In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('../data/data_titanic_2.csv',header='infer')
df.head()

,Embarked_Q,Embarked_S,Sex_male,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,SibSp_1,...,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Pclass_2,Pclass_3,"Age_(21.0, 30.0]","Age_(30.0, 35.0]","Age_(35.0, 80.0]",Survived
0,False,True,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,False,0
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,1
2,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,1
3,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,1
4,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,0


In [3]:
df.shape

(889, 21)

In [4]:
X = df.drop(columns=['Survived']) # variables independientes
y = df['Survived'] # variable dependiente

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Dividir los datos en conjunto de entrenamiento y prueba

In [6]:
model = DecisionTreeClassifier(random_state=42) # Crear el modelo de árbol de decisión
model.fit(X_train, y_train) # Entrenar el modelo
y_pred = model.predict(X_test) # Hacer predicciones

In [18]:
precision = accuracy_score(y_test, y_pred) # Calcular la precisión del modelo
print(f'Precisión del modelo: {precision*100:.2f}') # Mostrar la precisión del modelo
print('Matriz de confusión:')
print(confusion_matrix(y_test, y_pred)) # Mostrar la matriz de confusión
print('Reporte de clasificación:')
print(classification_report(y_test, y_pred)) # Mostrar el reporte de clasificación

Precisión del modelo: 79.78
Matriz de confusión:
[[95 14]
 [22 47]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       109
           1       0.77      0.68      0.72        69

    accuracy                           0.80       178
   macro avg       0.79      0.78      0.78       178
weighted avg       0.80      0.80      0.80       178



In [9]:
from sklearn.tree import export_text
reglas_arbol = export_text(model, feature_names=X.columns.tolist())
print("Reglas del árbol de decisión:")
print(reglas_arbol)

Reglas del árbol de decisión:
|--- Sex_male <= 0.50
|   |--- Pclass_3 <= 0.50
|   |   |--- Age_(35.0, 80.0] <= 0.50
|   |   |   |--- SibSp_1 <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- SibSp_1 >  0.50
|   |   |   |   |--- Age_(21.0, 30.0] <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- Age_(21.0, 30.0] >  0.50
|   |   |   |   |   |--- Pclass_2 <= 0.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Pclass_2 >  0.50
|   |   |   |   |   |   |--- Embarked_S <= 0.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- Embarked_S >  0.50
|   |   |   |   |   |   |   |--- Parch_1 <= 0.50
|   |   |   |   |   |   |   |   |--- Parch_2 <= 0.50
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- Parch_2 >  0.50
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- Parch_1 >  0.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |--- Age_(35.0, 80.0] >  0.50
|   |   |   |-

## Uso de Hiperparámetros

In [12]:
# Definición de rangos de hiperparámetros para la optimización (20x19x20x2 = 15200 combinaciones)
rango_max_depth = np.arange(1, 21) # Profundidad máxima del árbol, desde 1 hasta 20
rango_max_leaf_nodes = np.arange(2, 21)
rango_min_samples_leaf = np.arange(10, 31)
criterios = ['gini', 'entropy']
# Definición del diccionario de hiperparámetros
parametros = {
    'max_depth': rango_max_depth,
    'min_samples_leaf': rango_min_samples_leaf,
    'max_leaf_nodes': rango_max_leaf_nodes,
    'criterion': criterios
}

In [ ]:
# Obtener todas las combinaciones posibles de hiperparámetros

from sklearn.model_selection import GridSearchCV # Importar GridSearchCV. Esta clase permite realizar una búsqueda exhaustiva sobre un conjunto de hiperparámetros especificados.
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), parametros, cv=10, n_jobs=-1) # Crear el objeto GridSearchCV. cv=10 indica que se utilizará una validación cruzada de 10 pliegues. n_jobs=-1 permite utilizar todos los núcleos de la CPU disponibles para acelerar el proceso.
grid_search.fit(X_train, y_train) # Ajustar el modelo a los datos de entrenamiento

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                         'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20]),
                         'min_samples_leaf': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30])})

In [14]:
# Buscar la mejor combinación de hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params)
best_max_depth = best_params['max_depth']
best_min_samples_leaf = best_params['min_samples_leaf']
best_max_leaf_nodes = best_params['max_leaf_nodes']
best_criterion = best_params['criterion']
print(f"max_depth: {best_max_depth}, min_samples_leaf: {best_min_samples_leaf}, max_leaf_nodes: {best_max_leaf_nodes}, criterion: {best_criterion}")

Mejores hiperparámetros encontrados:
{'criterion': 'gini', 'max_depth': 3, 'max_leaf_nodes': 4, 'min_samples_leaf': 10}
max_depth: 3, min_samples_leaf: 10, max_leaf_nodes: 4, criterion: gini


In [17]:
# Entrenar el modelo con los mejores hiperparámetros
best_estimator = grid_search.best_estimator_ # Obtener el mejor estimador encontrado por GridSearchCV
print(best_estimator) # Mostrar el mejor estimador encontrado
y_pred_best = best_estimator.predict(X_test) # Hacer predicciones con el mejor estimador
precision_best = accuracy_score(y_test, y_pred_best) # Calcular la precisión del mejor estimador
print(f'Precisión del mejor modelo: {precision_best*100:.2f}%') # Mostrar la precisión del mejor modelo
print('Matriz de confusión del mejor modelo:')
print(confusion_matrix(y_test, y_pred_best)) # Mostrar la matriz de confusión del mejor modelo
print('Reporte de clasificación del mejor modelo:')
print(classification_report(y_test, y_pred_best)) # Mostrar el reporte de clasificación del mejor modelo

DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4, min_samples_leaf=10,
                       random_state=42)
Precisión del mejor modelo: 80.34%
Matriz de confusión del mejor modelo:
[[99 10]
 [25 44]]
Reporte de clasificación del mejor modelo:
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       109
           1       0.81      0.64      0.72        69

    accuracy                           0.80       178
   macro avg       0.81      0.77      0.78       178
weighted avg       0.80      0.80      0.80       178

